Previous: [pipeline_experiment](pipeline_experiment.ipynb) | Next: [pipeline_preprocess](pipeline_preprocess.ipynb)
# Schema `vis`
Schema `pipeline.vis` is upstream from `preprocess` and it contains information about the visual stimulus entered by the stimulus software.

#### Note that stimulus information used to be contained in the `psy` module.  It was moved to `vis` to isolate information relevant to the MICrONS project.

In [ ]:
%pylab inline
import datajoint as dj
from pipeline.vis import *
matplotlib.rcParams['figure.figsize'] = (10.0, 9.0)

Its three main tables are `vis.Session`, `vis.Condition`, and `vis.Trial`. Furthermore `vis.Condition` has many tables below specifying parameters specific to each type of stimulus condition.

In [ ]:
(dj.ERD(Condition)+1+Session+Trial).draw()


Each `vis.Session` comprises multiple trials that each has only one condition.  The trial has timing information and refers to the general stimulus condition.

---
The type of condition is determined by the dependent tables of `vis.Condition` (e.g. `vis.Monet`, `vis.Trippy`, `vis.MovieClipCond`) that describe details that are specific to each type of visual stimulus. Some of these tables have lookup tables with cached data-intensive stimuli such as noise movies.

In [ ]:
(dj.ERD(MovieClipCond)+Monet-1).draw()

Next: [pipeline_preprocess](pipeline_preprocess.ipynb)